In [1]:
%matplotlib qt 
# if you are using jupyter notebook/Lab the above command will make all the plots come out in a seperate dialog box instead of being embebed in the notebook.The sunpy_map.peek() function does not work properly without this.
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from glob import glob
from sunpy.map import Map
from datetime import datetime
import astropy.units as u
from astropy.coordinates import SkyCoord

# These are the packages used for coalignment. Visit  <https://docs.sunpy.org/projects/sunkit-image/en/latest/code_ref/coalignment.html>  to learn more.
from sunkit_image.coalignment import calculate_match_template_shift as cal_shift
from sunkit_image.coalignment import apply_shifts 

# to prep AIA level_1 data to level_1.5
from aiapy.calibrate import register, update_pointing
from aiapy.calibrate.util import get_pointing_table

from sunpy.coordinates import  get_horizons_coord
from sunpy.map import get_observer_meta

import warnings
# Suppress all SunPyUserWarnings
warnings.filterwarnings("ignore")
from copy import deepcopy

In [2]:
# This function takes your map sequence as input and returns coaligned maps as output.
def co_align(map_sequence,template,layer_index=0,clip=False):
    # template = user provided template which will be used to coalign the data
    # layer_index = aligns all the maps to this particular map in the map_sequence
    # clip =  If True, then clip off x, y edges of the maps in the sequence that are potentially affected by edges effects.
    
    shifts = cal_shift(map_sequence,layer_index=layer_index,template=template)
    plate_scale = map_sequence[0].scale[0]
    if clip is False:
        coaligned_maps = apply_shifts(map_sequence, xshift=-shifts['x']/plate_scale, yshift=-shifts['y']/plate_scale, clip=False)
    if clip is True:
        coaligned_maps = apply_shifts(map_sequence, xshift=-shifts['x']/plate_scale, yshift=-shifts['y']/plate_scale, clip=True)
    return coaligned_maps

In [3]:
# This function allows you to easily make template to coalign your map sequence. 
# This function make a submap of your input by asking you to select opposite corners of your desired submap in the input map.
def submap_with_ginput(sunpy_map):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection=sunpy_map)
    sunpy_map.plot(axes=ax)
    ax.set_title("Click two corners of the ROI (bottom-left and top-right)")

    # Wait for 2 clicks
    pts = plt.ginput(2, timeout=0)
    plt.close(fig)

    if len(pts) != 2:
        raise RuntimeError("ROI selection cancelled or failed.")

    (x1, y1), (x2, y2) = pts
    bottom_left = (min(x1, x2), min(y1, y2)) * u.pixel
    top_right = (max(x1, x2), max(y1, y2)) * u.pixel

    submap = sunpy_map.submap(bottom_left=bottom_left, top_right=top_right)
    return submap

In [4]:
# first load all SUIT images you want to coalign and plot. 
# We shall be using RoI images for this example. For more info on SUIT data visit - <https://link.springer.com/article/10.1007/s11207-025-02423-1>
files = sorted(glob('data/*fits'))
smaps = Map(files,sequence=True)
# smaps.peek()

In [5]:
# print index of each map. 
for i,tmap in enumerate(smaps):
    print(i,tmap.meta['ftr_name'])

0 NB04
1 NB03
2 NB02
3 NB05
4 NB06
5 NB07
6 NB08
7 NB01
8 BB02


 making a template to colaign the maps.
 Cautions while making a template; 
   1. Pick a region which does not change and appears similar in all the images in your map sequence
   2. Always pick a region which is smaller than the size of the map.
   3. Sunspots are a good choice if you are trying to coalign different filters as they will appear in all filters with fairly similar structure.

In [ ]:
# making a template using NB03 filter to align images
index_nb3 = 1
template = submap_with_ginput(smaps[index_nb3])
plt.figure()
template.plot()
# run this cell again if you want to change your template.

In [ ]:
# witness your miracle
coaligned_maps = co_align(smaps,template=template,layer_index=index_nb3)
coaligned_maps.peek()
# if the maps are not coaligned properly, go back to the previous cell and select a new template.
# Have Fun

In [ ]:
# # if your coalignment does not work for the data provided, try using this template.
# top_right = SkyCoord( -4* u.arcsec, -330 * u.arcsec, frame=smaps[index_nb3].coordinate_frame)
# bottom_left = SkyCoord(-63 * u.arcsec, -355 * u.arcsec, frame=smaps[index_nb3].coordinate_frame)
# template_2 = smaps[index_nb3].submap(bottom_left,top_right=top_right)
# template_2.plot()

In [ ]:
# to save the maps as png files on your local machine. where name = observation_time+filter_name.
# Change save path accordingly

for i,temp in enumerate(smaps):
    fig = plt.figure()
    ax = fig.add_subplot(projection=smaps[0])
    temp.plot(axes = ax,vmin=0,vmax=28000)
    ax.grid(False)
    plt.show()
    break
    plt.savefig(f"{temp.meta['dhobt_dt']}_{temp.meta['ftr_name']}.png",dpi=300)
    plt.close()

Now we shall try to align SUIT data with AIA observations

In [6]:
# alignning with AIA data
aia_data = Map(glob('aia_data/*.fits'),sequence=True)
# aia_data.peek()

In [7]:
# This function reprojects a SunPy map to a desired resolution (arcsec/pixel) by rotating/resampling, 
# ensures the map stays centered, crops back if oversized, fills gaps with a missing value, and updates
# the metadata with a pixel-scale solar radius.

def sun_register(smap,res=0.7* u.arcsec / u.pix, recenter=True, missing=None, order=3, method='scipy', clip=True):
    """
    res: Required resolution of the output maps. Default value: 0.7
    missing: Value to replace the empty array values. Default value: 0
    order: Default order of interpolation. Default order: 3
    method: Interpolation method to use. Default: 'scipy'
    """

    [x_shp, y_shp] = smap.data.shape

    if((smap.scale[0] / res).round() != 1.0 * u.arcsec / u.pix):
        if(smap.scale[0]/res>0.5):
            scale = (smap.scale[0] / res).round() * res * u.arcsec
        else:
            scale = (smap.scale[0]/res) * u.arcsec
    else:
        scale = res * u.arcsec

    scale_factor = smap.scale[0] / scale
    missing = smap.min() if missing is None else missing
    tempmap = smap.rotate(recenter=True,scale=scale_factor.value,order=order,missing=missing,method=method,clip=clip)
    center = np.floor(tempmap.meta["crpix1"])


    if(tempmap.data.shape[0]>x_shp):
        print('Here !!')
        range_side = (center + np.array([-1, 1]) * smap.data.shape[0] / 2) * u.pix
        newmap = tempmap.submap(u.Quantity([range_side[0], range_side[0]]),top_right=u.Quantity([range_side[1], range_side[1]]) - 1 * u.pix,)
    else:
        newmap = tempmap
    try:
        newmap.meta["r_sun"] = newmap.meta["rsun_obs"] / newmap.meta["cdelt1"]
        return newmap
    except KeyError:
        pass
        return newmap

In [8]:
# updating pointing info and registers the image so that all aia images are now coaligned.
nb3_index=1
aia_maps = []
for aia_map in aia_data:
    pointing_table = get_pointing_table("JSOC", time_range=(aia_map.date - 12 * u.h, aia_map.date + 12 * u.h))
    aia_map_updated_pointing = update_pointing(aia_map, pointing_table=pointing_table)
    aia_map_registered = register(aia_map_updated_pointing) #all the aia maps are now registered i.e coaligned
    
    #change the AIA's plate scale to SUIT plate scale
    aia_map_registered_to_suit_scale = sun_register(aia_map_registered,smaps[0].scale[0].value) 
    
    # crop the same region as in the SUIT Maps.
    aia_map_cropped = aia_map_registered.submap(smaps[nb3_index].bottom_left_coord,top_right=smaps[nb3_index].top_right_coord) 
    aia_maps.append(aia_map_cropped)
    
aia_maps = Map(aia_maps,sequence=True)
aia_maps.peek()

In [11]:
for temp in aia_maps:
    print(temp.meta['wavelnth'])

335
193
304
131
171
211
94
1600


In [16]:
plt.figure()
smaps[1].plot()
# aia1600.plot()

In [11]:
test = smaps[1]
aia1600 = deepcopy(aia_maps[-1])

t1 = test.meta['dhobt_dt']
suit_pos = get_horizons_coord("SOHO",t1)
test.meta.update(get_observer_meta(suit_pos,rsun=suit_pos.rsun))

test.meta['rsun_ref'] = aia1600.meta['rsun_ref']
#you can project it to any aia map since they are coaligned it is one and the same.
aia_projected_suitmap = test.reproject_to(aia1600.wcs,auto_extent='corners')

# # replace back headers
new_meta = deepcopy(test.meta)
new_meta.update(aia_projected_suitmap.meta)
projected_suitmap = Map(aia_projected_suitmap.data,new_meta)

2025-10-22 16:50:18 - sunpy - INFO: Obtained JPL HORIZONS location for SOHO (spacecraft) (-21)        
2025-10-22 16:50:18 - reproject.common - INFO: Calling _reproject_full in non-dask mode


INFO: Obtained JPL HORIZONS location for SOHO (spacecraft) (-21)         [sunpy.coordinates.ephemeris]


In [17]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6),subplot_kw={'projection': aia1600},
sharex=True, sharey=True)

# First subplot: AIA
aia1600.plot(axes=ax1)
ax1.set_title("AIA")

# Second subplot: SUIT projected on AIA WCS
aia_projected_suitmap.plot(axes=ax2) 
ax2.set_title("SUIT overlaid on AIA") 
plt.show()

In [80]:
template_3 = submap_with_ginput(temp[0])
plt.figure()
template_3.plot()

2025-10-01 17:40:49 - matplotlib.figure - INFO: input 1: 236.025974, 374.342532
2025-10-01 17:40:50 - matplotlib.figure - INFO: input 2: 324.913961, 327.061688


In [85]:
coaligned_temp = co_align(temp,template=template_3)
coaligned_temp.peek()